In [1]:
!pip install gym

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
from enum import Enum

In [3]:
def swap(state, action):  
    if action == 0: 
        row0, col0, row1, col1 = 0, 0, 0, 1
    elif action == 1: 
        row0, col0, row1, col1 = 0, 1, 0, 2
    elif action == 2:
        row0, col0, row1, col1 = 0, 0, 1, 0
    elif action == 3:
        row0, col0, row1, col1 = 0, 1, 1, 1
    elif action == 4:
        row0, col0, row1, col1 = 0, 2, 1, 2
    elif action == 5:
        row0, col0, row1, col1 = 1, 0, 1, 1
    elif action == 6:
        row0, col0, row1, col1 = 1, 1, 1, 2
    elif action == 7:
        row0, col0, row1, col1 = 1, 0, 2, 0
    elif action == 8:
        row0, col0, row1, col1 = 1, 1, 2, 1
    elif action == 9:
        row0, col0, row1, col1 = 1, 2, 2, 2
    elif action == 10:
        row0, col0, row1, col1 = 2, 0, 2, 1
    elif action == 11:
        row0, col0, row1, col1 = 2, 1, 2, 2
        
    #swap for all future states
    for i in range(len(state)):
        state[i][row0][col0], state[i][row1][col1] = state[i][row1][col1], state[i][row0][col0]

In [4]:
def getNeighbors(state, row_number, column_number):
    a = [[state[i][j] if  i >= 0 and i < len(state) and j >= 0 and j < len(state[0]) else 0
                    for j in range(column_number-1, column_number+2)]
                        for i in range(row_number-1, row_number+2)]
    return [a[0][1], a[1][0], a[1][2], a[2][1]]

In [5]:
#                      [[1,0,0],
#Takes in a state like  [1,0,2], and checks if the corresonding numbers are neighbors
#                       [2,0,0]]

def isExecutableState(state):
    for row in range(len(state[0])):
        for col in range(len(state[0][0])):
            if state[0][row][col] != 0:
                if not state[0][row][col] in getNeighbors(state[0], row, col):
                    return False
    #return bool
    return True

testing of the functions isValidState and swap

In [6]:
arr = np.zeros((10,3,3), dtype=int)
arr[0][0][1] = 1
arr[0][0][2] = 2
arr[0][1][1] = 1
arr[0][1][2] = 2
arr[1] = arr[0]

print(arr[0])
print(arr[1])
print(isExecutableState(arr))

swap(arr, 1)
print(arr[0])
print(arr[1])
print(isExecutableState(arr))

[[0 1 2]
 [0 1 2]
 [0 0 0]]
[[0 1 2]
 [0 1 2]
 [0 0 0]]
True
[[0 2 1]
 [0 1 2]
 [0 0 0]]
[[0 2 1]
 [0 1 2]
 [0 0 0]]
False


In [7]:
def makeState():
    state = np.zeros((10,3,3))
    for i in range(len(state)):
        random = np.random.choice([3,5,7,9])
        matrix = np.ceil(np.arange(random)/2)
        matrix = np.append(matrix, np.zeros(9-random)).reshape((3,3))
        np.random.shuffle(matrix)
        state[i] = matrix
    return state

Test function makeState

In [8]:
makeState()

array([[[0., 0., 0.],
        [0., 1., 1.],
        [0., 0., 0.]],

       [[0., 1., 1.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 1., 1.],
        [0., 0., 0.]],

       [[0., 1., 1.],
        [2., 2., 0.],
        [0., 0., 0.]],

       [[2., 2., 3.],
        [3., 0., 0.],
        [0., 1., 1.]],

       [[0., 0., 0.],
        [2., 2., 0.],
        [0., 1., 1.]],

       [[0., 1., 1.],
        [2., 2., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [2., 2., 0.],
        [0., 1., 1.]],

       [[3., 4., 4.],
        [2., 2., 3.],
        [0., 1., 1.]],

       [[0., 1., 1.],
        [0., 0., 0.],
        [0., 0., 0.]]])

In [9]:
#Our enviorment
class Kvant(Env):
    def __init__(self):
        #Actions we can take
        self.action_space = Discrete(12)
        
        #
        self.observation_space = Box(low=np.zeros((10,3,3), dtype=int), high=np.zeros((10,3,3), dtype=int)+4)
        
        #The start state
        self.state = makeState()
        
        #timesteps
        self.time_steps = 10
        
    def step(self, action):
        swap(self.state, action)
        
        self.time_steps -= 1
        
        if isExecutableState(self.state):
            reward = 1
        elif self.time_steps <= 0:
            reward = -1
        else:
            reward = -1
            
    
        if isExecutableState(self.state) or self.time_steps <= 0:
            done = True
        else:
            done = False
        info = {}
        
        return self.state, reward, done, info
        
    def render(self):
        pass
    def reset(self):
        self.state = makeState()
        self.time_steps = 10
        return self.state

In [10]:
env = Kvant()

/home/surface/.local/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [11]:
episodes = 1000
scores = []
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    scores.append(score)

In [12]:
for i in range(-100, 2):
    if scores.count(i) != 0:
        print('Number of', i, 'is', scores.count(i))

Number of -10 is 506
Number of -8 is 7
Number of -7 is 8
Number of -6 is 21
Number of -5 is 17
Number of -4 is 22
Number of -3 is 25
Number of -2 is 21
Number of -1 is 33
Number of 0 is 49
Number of 1 is 291
